In [44]:
import os
from langchain.chat_models import ChatOpenAI
from pdfquery import PDFQuery

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
import json

In [45]:
##immport data
with open('productos.json', 'r') as file:
    loaded_list = json.load(file)
    print(loaded_list)

['"Kit x 2 Frozen de Disney: Gel para el Cabello 65 g, precio g $187,24 + Colonia 70 ml. Precio regular $44.800 (168391) 2 pts KIT X 2 $21.900"', '"Gel para el Cabello Frozen de Disney 65 g. Precio regular $24.900 (133996) 1 pt $13.900 Precio g $213,85"', '"Colonia Frozen de Disney 70 ml. Precio regular $19.900 (133993) 1 pt $12.900"', '"Cepillo Corona Diamantes Frozen de Disney: Plástico y nailon. Diseño con forma de corona con simulación de diamantes. Largo: 17.6 cm, ancho: 7.5 cm, alto: 2.5 cm. Peso: 44 g. Precio regular $34.900 (133122) 2 pts $22.900"', '"Shampoo 2 en 1 Frozen de Disney 200 ml. Precio regular $27.900 (133995) 1 pt $16.900 Precio ml $84,50"', '"Barrita Humectante Frozen de Disney 4.5 g. Precio regular $19.900 (133994) 1 pt $9.900"', '"Colonia Encanto de Disney 150 ml. Precio regular $25.900 (133669) 2 pts $18.900"', '"Shampoo 2 en 1 Encanto de Disney 200 ml. Precio regular $27.900 (133670) 1 pt $16.900 Precio ml $84,50"', '"Crema para Peinar Encanto de Disney 150 g.

In [46]:
# embed and index all data
from pinecone import Pinecone
from pinecone import ServerlessSpec

# initialize connection to pinecone (get API key at app.pinecone.io)
#api_key = os.getenv("PINECONE_API_KEY") or "YOUR_API_KEY"

# configure client
pc = Pinecone(api_key= "API-KEY")

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

chat = ChatOpenAI(
    openai_api_key="API-KEY",
    model='gpt-3.5-turbo'
)

In [47]:
import time

index_name = 'llama-2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 38}},
 'total_vector_count': 38}

In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002") ##use this model to create embeddings 

##create embedding for the dataset
from tqdm.auto import tqdm  # for progress bar

batch_size = 100

for i in tqdm(range(0, len(loaded_list), batch_size)):
    i_end = min(len(loaded_list), i+batch_size)
    # get batch of data
    batch = loaded_list[i:i_end]
    # generate unique ids for each chunk
    ids = list(range(i,i_end))
    # Convert the list of integers to a list of strings
    ids_str = [str(id) for id in ids]
    # embed text
    embeds = embed_model.embed_documents(batch)

    #metadata
    metadata = [{'text': x } for x in batch]
    print(metadata)
    # add to Pinecone
    index.upsert(vectors=zip(ids_str, embeds, metadata))


  0%|          | 0/1 [00:00<?, ?it/s]

[{'text': '"Kit x 2 Frozen de Disney: Gel para el Cabello 65 g, precio g $187,24 + Colonia 70 ml. Precio regular $44.800 (168391) 2 pts KIT X 2 $21.900"'}, {'text': '"Gel para el Cabello Frozen de Disney 65 g. Precio regular $24.900 (133996) 1 pt $13.900 Precio g $213,85"'}, {'text': '"Colonia Frozen de Disney 70 ml. Precio regular $19.900 (133993) 1 pt $12.900"'}, {'text': '"Cepillo Corona Diamantes Frozen de Disney: Plástico y nailon. Diseño con forma de corona con simulación de diamantes. Largo: 17.6 cm, ancho: 7.5 cm, alto: 2.5 cm. Peso: 44 g. Precio regular $34.900 (133122) 2 pts $22.900"'}, {'text': '"Shampoo 2 en 1 Frozen de Disney 200 ml. Precio regular $27.900 (133995) 1 pt $16.900 Precio ml $84,50"'}, {'text': '"Barrita Humectante Frozen de Disney 4.5 g. Precio regular $19.900 (133994) 1 pt $9.900"'}, {'text': '"Colonia Encanto de Disney 150 ml. Precio regular $25.900 (133669) 2 pts $18.900"'}, {'text': '"Shampoo 2 en 1 Encanto de Disney 200 ml. Precio regular $27.900 (133670

In [49]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 38}},
 'total_vector_count': 38}

In [50]:
# Retrieval Augmented Generation

from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [60]:
query = "geles"

vectorstore.similarity_search(query, k=5)

[Document(page_content='"Colonia Encanto de Disney 150 ml. Precio regular $25.900 (133669) 2 pts $18.900"'),
 Document(page_content='"Colonias Disney Princesa 150 ml. Precio regular $25.900 / 2 pts (139188) Ariel, (139189) Blanca Nieves, (139191) Rapunzel"'),
 Document(page_content='"Colonia para Niños Avon Universo 150 ml. Precio regular $19.900 (130595) 1 pt"'),
 Document(page_content='"Crema para Peinar Encanto de Disney 150 g. Precio regular $25.900 (133671) 1 pt $16.900 Precio g $112,67"'),
 Document(page_content='"Gel para el Cabello Frozen de Disney 65 g. Precio regular $24.900 (133996) 1 pt $13.900 Precio g $213,85"')]

In [57]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=5)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""
    Usando los contextos a continuación, responde la consulta. Todas las preguntas deben ser sobre productos, precios y descuentos en el contexto, o productos para baño y aseo. 

    Contextos:
    {source_knowledge}

    Consulta: {query}
    """

    return augmented_prompt

In [63]:
query = 'tienes productos por menos de $20.000?'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Sí, tenemos colonias para niños Avon Universo y Avon Kids Safari de 150 ml a un precio regular de $19.900.


In [62]:
query = 'que geles tienes?'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Tenemos disponibles los siguientes geles:
1. Gel para el Cabello Frozen de Disney 65 g. Precio regular $24.900 (133996) 1 pt $13.900 Precio g $213,85
2. Gel para el Cabello Avengers de Marvel 100 g. Precio regular $24.900 (130577) 1 pt $15.900 Precio g $159,00
3. Gel para el Cabello Spider-Man de Marvel 100 g. Precio regular $24.900 (130609) 1 pt $13.900 Precio g $139,00


In [38]:
query = 'sabes sobre la historia de Colombia?'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Lo siento, no tengo información sobre la historia de Colombia. ¿Tienes alguna otra pregunta relacionada con productos, precios o descuentos?


In [39]:
query = 'Quien escribio el libro "El Tunel"'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Lo siento, esa pregunta no está relacionada con los contextos proporcionados. ¿Tienes alguna otra pregunta sobre productos, precios y descuentos en el contexto, o productos para baño y aseo personas?


In [55]:
query = 'que tienes para el cabello'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Tenemos diferentes opciones para el cabello, como la Crema para Peinar Barbie, el Gel para el Cabello Avon Kids Safari, el Gel para el Cabello Avengers de Marvel, el Gel para el Cabello Spider-Man de Marvel y la Crema para Peinar Mérida de Disney Princesa. ¿Te interesa alguno en particular?


In [64]:
query = 'tienes cremas para el cuerpo'
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages = [
    #SystemMessage(content="You are a helpful assistant."),
    prompt
    #HumanMessage(content=prompt)
]

res = chat(messages)

print(res.content)

Lo siento, en este contexto solo tenemos información sobre cremas para peinar y colonias para niños. No disponemos de información sobre cremas para el cuerpo en este momento. ¿Hay algo más en lo que pueda ayudarte?
